```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
import os
import sys
import pyopencl
from matplotlib import animation, rc
from matplotlib import pyplot as plt

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, LxF, HLL, HLL2, KP07, DataOutput, PlotHelper

In [2]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

Using  Intel(R) HD Graphics Skylake ULT GT2


In [3]:
# Forward backward linear

nx = 100
ny = 200

dx = 20.0
dy = 20.0

dt = 0.1
g = 9.81
f = 0.0
r = 0.0

h0 = np.ones((ny, nx), dtype=np.float32, order='C') * 5;
eta0 = np.zeros((ny, nx), dtype=np.float32, order='C');
u0 = np.zeros((ny, nx+1), dtype=np.float32, order='C');
v0 = np.zeros((ny+1, nx), dtype=np.float32, order='C');

#Create bump in center of domain for testing
x_center = dx*nx/2.0
y_center = dy*ny/2.0
for j in range(ny):
    for i in range(nx):
        x = dx*i - x_center
        y = dy*j - y_center
        size = 500.0*min(dx, dy)
        if (np.sqrt(x**2 + y**2) < size):
            eta0[j, i] = np.exp(-(x**2/size+y**2/size))
            
#Initialize simulator
reload(FBL)
sim = FBL.FBL(cl_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0, u0, v0)


def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1, u1, v1);
    fig.suptitle("FBL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t)
        fig.savefig("{:010.0f}_fbl.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim

000 % => t=0.0
000 % => t=0.0
020 % => t=99.999
040 % => t=200.003
060 % => t=300.009
080 % => t=400.015


In [4]:
#Centered in time, centered in space

nx = 100
ny = 200

dx = 20.0
dy = 20.0

dt = 0.1
g = 9.81
f = 0.0
r = 0.0
A = 1

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 5;
eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u0 = np.zeros((ny+2, nx+1), dtype=np.float32, order='C');
v0 = np.zeros((ny+1, nx+2), dtype=np.float32, order='C');

#Create bump in center of domain for testing
x_center = dx*nx/2.0
y_center = dy*ny/2.0
for j in range(-1, ny+1):
    for i in range(-1, nx+1):
        x = dx*i - x_center 
        y = dy*j - y_center
        size = 500.0*min(dx, dy)
        if (np.sqrt(x**2 + y**2) < size):
            eta0[j+1, i+1] = np.exp(-(x**2/size+y**2/size))
            
#Initialize simulator
reload(CTCS)
sim = CTCS.CTCS(cl_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0[1:-1, 1:-1], u0[1:-1, :], v0[:, 1:-1])


def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[1:-1, 1:-1], u1[1:-1, :], v1[:, 1:-1]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t)
        fig.savefig("{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim

/usr/lib/python2.7/dist-packages/pyopencl/__init__.py:59: CompilerWarning: From-source build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) HD Graphics Skylake ULT GT2' on 'Intel Gen OCL Driver' at 0x7f84b04d22c0> succeeded, but said:

stringInput.cl:185:21: warning: double precision constant requires cl_khr_fp64, casting to single precision
                                                                                                            
  warn(text, CompilerWarning)
/usr/lib/python2.7/dist-packages/pyopencl/__init__.py:59: CompilerWarning: From-source build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) HD Graphics Skylake ULT GT2' on 'Intel Gen OCL Driver' at 0x7f84b04d22c0> succeeded, but said:

stringInput.cl:189:21: warning: double precision constant requires cl_khr_fp64, casting to single precision
                                                                                                            


000 % => t=0.0
000 % => t=0.0
020 % => t=99.999
040 % => t=200.003
060 % => t=300.009
080 % => t=400.015


In [3]:
# Lax Friedrichs
reload(LxF)

nx = 100
ny = 200

dx = 20.0
dy = 20.0

dt = 1.5
g = 9.81

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 5;
u0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
v0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');

#Create bump in center of domain for testing
x_center = dx*nx/2.0
y_center = dy*ny/2.0
for j in range(-1, ny+1):
    for i in range(-1, nx+1):
        x = dx*i - x_center
        y = dy*j - y_center
        size = 500.0*min(dx, dy)
        if (np.sqrt(x**2 + y**2) < size):
            h0[j+1, i+1] += np.exp(-(x**2/size+y**2/size))
            
#Initialize simulator
reload(LxF)
sim = LxF.LxF(cl_ctx, \
                h0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, h0[1:-1, 1:-1] - 5, u0[1:-1, 1:-1], v0[1:-1, 1:-1])


def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    plotter.plot(h1[1:-1, 1:-1] - 5.0, u1[1:-1, 1:-1], v1[1:-1, 1:-1]);
    fig.suptitle("LxF Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t)
        fig.savefig("{:010.0f}_lxf.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim

000 % => t=0.0
000 % => t=0.0
020 % => t=100.0
040 % => t=200.0
060 % => t=300.0
080 % => t=400.0


In [6]:
# Kurganov-Petrova 2007 paper
reload(KP07)

nx = 100
ny = 200

dx = 20.0
dy = 20.0

dt = 0.9
g = 9.81
f = 0.0
r = 0.0

h0 = np.ones((ny+4, nx+4), dtype=np.float32, order='C') * 5;
u0 = np.zeros((ny+4, nx+4), dtype=np.float32, order='C');
v0 = np.zeros((ny+4, nx+4), dtype=np.float32, order='C');

#Create bump in center of domain for testing
x_center = dx*nx/2.0
y_center = dy*ny/2.0
for j in range(-2, ny+2):
    for i in range(-2, nx+2):
        x = dx*i - x_center
        y = dy*j - y_center
        size = 500.0*min(dx, dy)
        if (np.sqrt(x**2 + y**2) < size):
            h0[j+2, i+2] += np.exp(-(x**2/size+y**2/size))
            
#Initialize simulator
reload(KP07)
sim = KP07.KP07(cl_ctx, \
                h0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, h0[2:-2, 2:-2] - 5, u0[2:-2, 2:-2], v0[2:-2, 2:-2])


def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    plotter.plot(h1[2:-2, 2:-2] - 5.0, u1[2:-2, 2:-2], v1[2:-2, 2:-2]);
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t)
        fig.savefig("{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim

000 % => t=0.0
000 % => t=0.0
020 % => t=100.0
040 % => t=200.0
060 % => t=299.999
080 % => t=399.998


In [7]:
# Harten-Lax-van Leer
nx = 100
ny = 200

dx = 20.0
dy = 20.0

dt = 1.5
g = 9.81

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 5;
u0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
v0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');

#Create bump in center of domain for testing
x_center = dx*nx/2.0
y_center = dy*ny/2.0
for j in range(-1, ny+1):
    for i in range(-1, nx+1):
        x = dx*i - x_center
        y = dy*j - y_center
        size = 500.0*min(dx, dy)
        if (np.sqrt(x**2 + y**2) < size):
            h0[j+1, i+1] += np.exp(-(x**2/size+y**2/size))
            
#Initialize simulator
reload(HLL)
sim = HLL.HLL(cl_ctx, \
                h0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, h0[1:-1, 1:-1] - 5, u0[1:-1, 1:-1], v0[1:-1, 1:-1])


def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    plotter.plot(h1[1:-1, 1:-1] - 5.0, u1[1:-1, 1:-1], v1[1:-1, 1:-1]);
    fig.suptitle("HLL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t)
        fig.savefig("{:010.0f}_hll.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim

000 % => t=0.0
000 % => t=0.0
020 % => t=100.0
040 % => t=200.0
060 % => t=300.0
080 % => t=400.0


In [8]:
#HLL with 2nd order slope limited (minmod) reconstruction
nx = 100
ny = 200

dx = 20.0
dy = 20.0

dt = 0.9
g = 9.81

h0 = np.ones((ny+4, nx+4), dtype=np.float32, order='C') * 5;
u0 = np.zeros((ny+4, nx+4), dtype=np.float32, order='C');
v0 = np.zeros((ny+4, nx+4), dtype=np.float32, order='C');

#Create bump in center of domain for testing
x_center = dx*nx/2.0
y_center = dy*ny/2.0
for j in range(-2, ny+2):
    for i in range(-2, nx+2):
        x = dx*i - x_center
        y = dy*j - y_center
        size = 500.0*min(dx, dy)
        if (np.sqrt(x**2 + y**2) < size):
            h0[j+2, i+2] += np.exp(-(x**2/size+y**2/size))
            
#Initialize simulator
reload(HLL2)
sim = HLL2.HLL2(cl_ctx, \
                h0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, h0[2:-2, 2:-2] - 5, u0[2:-2, 2:-2], v0[2:-2, 2:-2])


def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    plotter.plot(h1[2:-2, 2:-2] - 5.0, u1[2:-2, 2:-2], v1[2:-2, 2:-2]);
    fig.suptitle("HLL2 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t)
        fig.savefig("{:010.0f}_hll2.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim

000 % => t=0.0
000 % => t=0.0
020 % => t=100.0
040 % => t=200.0
060 % => t=299.999
080 % => t=399.998
